In [41]:
#!pip install bertopic
#!pip install openai

In [42]:
import pandas as pd
import time
from matplotlib import pyplot as plt
from datetime import datetime
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from umap import UMAP
import re

In [43]:
def save(df):
    ts = int(time.time())
    df.to_excel("./files/{}.xlsx".format(ts))
    print(str(ts)+".xlsx has been saved")

In [44]:
path = "https://raw.githubusercontent.com/ayamlearning/masters_thesis_model_final/main/data/sample.csv"
df = pd.read_csv(path,delimiter=",",dtype=str)
print(df.shape)

(21098, 33)


In [45]:
df.columns

Index(['id', 'tweet_id', 'username', 'followers_count', 'user_verified',
       'text', 'created_at', 'like_count', 'retweet_count', 'reply_count',
       'location', 'lang', 'generation', 'ass_hashtags', 'keyword_exists',
       'prep_text', 'year', 'month', 'day', 'week', 'time', 'hour', 'minute',
       'vader_polarity', 'bert_polarity', 'date', 'city', 'district',
       'province', 'country', 'quater', 'weekday', 'tweet_lang'],
      dtype='object')

In [46]:
df.sample(5)

,id,tweet_id,username,followers_count,user_verified,text,created_at,like_count,retweet_count,reply_count,...,vader_polarity,bert_polarity,date,city,district,province,country,quater,weekday,tweet_lang
19201,71123,1067867666751328259,ZambiaElections,37562.0,False,The #ECZStrategicPlan2018to2022 was developed...,2018-11-28 21:47:46+02:00,2.0,1.0,0.0,...,Neutral,Neutral,2018-11-28 21:47:46+02:00,Lusaka,Lusaka District,Lusaka Province,Zambia,4,Wednesday,en
12630,64102,1379811228298674178,Space2013M,3762.0,False,@KalunduMan @ZambiaElections Imagine. How do p...,2021-04-07 17:00:07+02:00,1.0,0.0,0.0,...,Neutral,Negative,2021-04-07 17:00:07+02:00,NaN,NaN,NaN,NaN,2,Wednesday,en
12811,64299,1377111560271798279,DiggersOfNews,111280.0,False,Zambia unlikely to clinch IMF deal before elec...,2021-03-31 06:12:36+02:00,6.0,1.0,0.0,...,Neutral,Neutral,2021-03-31 06:12:36+02:00,NaN,NaN,NaN,NaN,1,Wednesday,en
11906,63349,1400159788735123458,gkaluba,161.0,False,@ZambiaElections @mthoniswa Most likely offici...,2021-06-02 20:38:02+02:00,3.0,1.0,0.0,...,Neutral,Neutral,2021-06-02 20:38:02+02:00,Kitwe,Kitwe District,Copperbelt Province,Zambia,2,Wednesday,en
12459,63920,1385695372841005057,misheck_cheleka,6650.0,False,@ConOfficial044 @ZambiaAsks @Mafken_kanjebwe @...,2021-04-23 22:41:36+02:00,0.0,0.0,0.0,...,Positive,Neutral,2021-04-23 22:41:36+02:00,NaN,NaN,NaN,NaN,2,Friday,en




```
# This is formatted as code
```

#Neutral Sentiments

In [47]:
df_query = df.query("vader_polarity == '{}'".format("Neutral"))
df_query.sample(5)

,id,tweet_id,username,followers_count,user_verified,text,created_at,like_count,retweet_count,reply_count,...,vader_polarity,bert_polarity,date,city,district,province,country,quater,weekday,tweet_lang
9632,60904,1425399500760354817,Kalemba_News,980.0,False,PVT allowed but no announcement of results - E...,2021-08-11 12:11:38+02:00,0.0,0.0,0.0,...,Neutral,Neutral,2021-08-11 12:11:38+02:00,Lusaka,Lusaka District,Lusaka Province,Zambia,3,Wednesday,en
149,10974,1425524099502198785,TeeMak9,1982.0,False,The people camping the night #ZambiaDecides2021,2021-08-11 20:26:44+02,0.0,0.0,0.0,...,Neutral,Neutral,2021-08-11 20:26:44+02:00,NaN,NaN,NaN,NaN,3,Wednesday,en
16863,68636,1242077211106119683,ZambiaElections,37562.0,False,Please be advised that the Poll for the By-ele...,2020-03-23 15:14:18+02:00,1.0,0.0,2.0,...,Neutral,Neutral,2020-03-23 15:14:18+02:00,Lusaka,Lusaka District,Lusaka Province,Zambia,1,Monday,en
19009,70922,1096656841373872128,ZambiaSoccerNet,9708.0,False,"In Zambia, it is unheard of to have a serving ...",2019-02-16 08:25:40+02:00,1.0,0.0,1.0,...,Neutral,Neutral,2019-02-16 08:25:40+02:00,NaN,NaN,NaN,Zambia,1,Saturday,en
16556,68249,1268105529433436160,MonicaEisenberg,10776.0,False,@VictorKanyense @BandaBunny @EdgarCLungu @HHic...,2020-06-03 11:01:33+02:00,1.0,1.0,2.0,...,Neutral,Negative,2020-06-03 11:01:33+02:00,NaN,NaN,NaN,NaN,2,Wednesday,en


In [48]:
df_query.text = df_query.apply(lambda row: re.sub(r"http\S+", "", row.text).lower(), 1)
df_query.text = df_query.apply(lambda row: " ".join(filter(lambda x:x[0]!="@", row.text.split())), 1)
df_query.text = df_query.apply(lambda row: " ".join(re.sub("[^a-zA-Z]+", " ", row.text).split()), 1)
timestamps = df_query.date.to_list()
tweets = df_query.text.to_list()

In [49]:
docs = list(map(str,  df_query['prep_text']))
topic_model = BERTopic(language="english", calculate_probabilities=True, verbose=True)
topics, probs = topic_model.fit_transform(docs)

Batches:   0%|          | 0/406 [00:00<?, ?it/s]

2023-10-21 07:26:52,834 - BERTopic - Transformed documents to Embeddings
2023-10-21 07:27:09,041 - BERTopic - Reduced dimensionality
2023-10-21 07:27:56,278 - BERTopic - Clustered reduced embeddings


In [50]:
topic_model.visualize_barchart()

In [74]:
topics_over_time = topic_model.topics_over_time(tweets, timestamps, nr_bins=20)
tt =topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=8)
tt

20it [00:01, 12.49it/s]


In [52]:
sentence_model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = sentence_model.encode(docs, show_progress_bar=False)
reduced_embeddings = UMAP(n_neighbors=10, n_components=2, min_dist=0.0, metric='cosine').fit_transform(embeddings)
topic_model.visualize_documents(docs, reduced_embeddings=reduced_embeddings)

In [53]:
topic_model.visualize_hierarchy(top_n_topics=7)

In [54]:
# Calculate the topic distributions on a token-level
topic_distr, topic_token_distr = topic_model.approximate_distribution(docs, calculate_tokens=True)

# Visualize the token-level distributions
topic_model.visualize_approximate_distribution(docs[1], topic_token_distr[1])

100%|██████████| 13/13 [00:02<00:00,  4.38it/s]


,day,left,remember,keep,best,attitude,avoid,getting,conflict,law,vote,count,visiting
11_day_remaining_till_left,0.240,0.370,0.370,0.370,0.130,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
90_law_lawyer_procedure_studying,0.000,0.000,0.000,0.000,0.000,0.000,0.116,0.253,0.394,0.529,0.412,0.275,0.134
97_till_day_13hrs_snub,0.153,0.153,0.153,0.153,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
106_voice_alot_micomyizajohn_automatically,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.154,0.154,0.154,0.154
120_wire_ku_left_ka,0.186,0.326,0.326,0.326,0.140,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
124_remember_booth_remembers_shot,0.313,0.543,0.719,0.719,0.406,0.176,0.000,0.000,0.000,0.000,0.000,0.000,0.000
128_decides_day_30_watermelon,0.115,0.115,0.115,0.115,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
242_internet_keep_switching_elearning,0.262,0.459,0.610,0.749,0.487,0.290,0.139,0.000,0.000,0.000,0.000,0.000,0.000


In [55]:
pd.DataFrame(topic_model.get_topic(0),columns=["Word","Score"])

,Word,Score
0,calendar,0.016112
1,general,0.014975
2,campaign,0.012508
3,2021,0.012408
4,election,0.011541
5,august,0.010518
6,checklist,0.009611
7,12,0.009488
8,event,0.008524
9,12th,0.008123


## **Negative** Sentiments

In [56]:
df_query = df.query("vader_polarity == '{}'".format("Negative"))
df_query.sample(5)

,id,tweet_id,username,followers_count,user_verified,text,created_at,like_count,retweet_count,reply_count,...,vader_polarity,bert_polarity,date,city,district,province,country,quater,weekday,tweet_lang
5816,16366,1303806565502377989,Peter_S_Becks94,1476.0,False,@Ch0nya @Kubale12 @chola_neene @MonicaEisenber...,2020-09-09 23:24:43+02:00,0.0,0.0,0.0,...,Negative,Negative,2020-09-09 23:24:43+02:00,NaN,NaN,NaN,Zambia,3,Wednesday,en
14615,66223,1322131392491360257,ercafrica,44427.0,False,Zambia Parliament reject Amendment Bill No.10 ...,2020-10-30 13:01:02+02:00,3.0,1.0,2.0,...,Negative,Negative,2020-10-30 13:01:02+02:00,NaN,Harare,Harare Province,Zimbabwe,4,Friday,en
13233,64747,1364931611997966336,2021Zambian,2714.0,False,Evidence from @ssishuwa on how @OfficialECZ Vo...,2021-02-25 15:33:50+02:00,2.0,2.0,0.0,...,Negative,Negative,2021-02-25 15:33:50+02:00,NaN,NaN,NaN,NaN,1,Thursday,en
15878,67554,1294333148164227072,netbuzzafrica,2218.0,False,"Lockdown killings, delayed elections, &amp; a ...",2020-08-14 20:00:44+02:00,1.0,0.0,0.0,...,Negative,Negative,2020-08-14 20:00:44+02:00,NaN,NaN,NaN,Africa,3,Friday,en
15742,67418,1300822056695717888,Peter_S_Becks94,1477.0,False,@VanguardAfrica @UPNDZM @HHichilema @Smith_Jef...,2020-09-01 17:45:21+02:00,0.0,0.0,1.0,...,Negative,Negative,2020-09-01 17:45:21+02:00,NaN,NaN,NaN,Zambia,3,Tuesday,en


In [57]:
df_query.text = df_query.apply(lambda row: re.sub(r"http\S+", "", row.text).lower(), 1)
df_query.text = df_query.apply(lambda row: " ".join(filter(lambda x:x[0]!="@", row.text.split())), 1)
df_query.text = df_query.apply(lambda row: " ".join(re.sub("[^a-zA-Z]+", " ", row.text).split()), 1)
timestamps = df_query.date.to_list()
tweets = df_query.text.to_list()

In [58]:
docs = list(map(str,  df_query['prep_text']))
topic_model = BERTopic(language="english", calculate_probabilities=True, verbose=True)
topics, probs = topic_model.fit_transform(docs)

Batches:   0%|          | 0/94 [00:00<?, ?it/s]

2023-10-21 07:31:21,247 - BERTopic - Transformed documents to Embeddings
2023-10-21 07:31:43,626 - BERTopic - Reduced dimensionality
2023-10-21 07:31:44,162 - BERTopic - Clustered reduced embeddings


In [59]:
topic_model.visualize_barchart()

In [60]:
topics_over_time = topic_model.topics_over_time(tweets, timestamps, nr_bins=20)
topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=8)

20it [00:00, 22.73it/s]


In [61]:
sentence_model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = sentence_model.encode(docs, show_progress_bar=False)
reduced_embeddings = UMAP(n_neighbors=10, n_components=2, min_dist=0.0, metric='cosine').fit_transform(embeddings)
topic_model.visualize_documents(docs, reduced_embeddings=reduced_embeddings)

In [62]:
topic_model.visualize_hierarchy(top_n_topics=7)

In [63]:
# Calculate the topic distributions on a token-level
topic_distr, topic_token_distr = topic_model.approximate_distribution(docs, calculate_tokens=True)

# Visualize the token-level distributions
topic_model.visualize_approximate_distribution(docs[1], topic_token_distr[1])

100%|██████████| 3/3 [00:00<00:00,  3.84it/s]


,disappointing,see,zambia,falling,low


In [64]:
pd.DataFrame(topic_model.get_topic(0),columns=["Word","Score"])

,Word,Score
0,zambia,0.030699
1,vote,0.020374
2,people,0.019657
3,zambian,0.015611
4,country,0.015428
5,poverty,0.015344
6,election,0.014609
7,cry,0.013582
8,economy,0.013276
9,poor,0.013052


## POSTIVE Sentiments

In [65]:
df_query = df.query("vader_polarity == '{}'".format("Positive"))
df_query.sample(5)

,id,tweet_id,username,followers_count,user_verified,text,created_at,like_count,retweet_count,reply_count,...,vader_polarity,bert_polarity,date,city,district,province,country,quater,weekday,tweet_lang
20166,72147,962017104269516800,chr779,1298.0,False,This looks familiar #Zambia 2016 elections😄 ht...,2018-02-09 19:35:25+02:00,0.0,0.0,0.0,...,Positive,Neutral,2018-02-09 19:35:25+02:00,NaN,NaN,NaN,NaN,1,Friday,en
3316,48935,1400031619898019847,mangalita5,2039.0,False,"Grace Namunyola, is standing as parliamentary ...",2021-06-02 12:08:44+02,2.0,1.0,1.0,...,Positive,Neutral,2021-06-02 12:08:44+02:00,NaN,NaN,NaN,NaN,2,Wednesday,en
3696,11013,1425491853617647621,SophieMwanza3,191.0,False,All the best tomorrow \n#ZambiaDecides2021 htt...,2021-08-11 18:18:36+02:00,1.0,0.0,0.0,...,Positive,Positive,2021-08-11 18:18:36+02:00,NaN,NaN,NaN,NaN,3,Wednesday,en
17272,69078,1227312736465453059,Zumani_,602.0,False,Great work @ZambiaElections! Praying for a pea...,2020-02-11 21:25:33+02:00,0.0,0.0,0.0,...,Positive,Positive,2020-02-11 21:25:33+02:00,Lusaka,Lusaka District,Lusaka Province,Zambia,1,Tuesday,en
17304,69114,1226558281331638272,MKapelembe,88.0,False,@normanchips @ZambiaElections @EdgarCLungu I p...,2020-02-09 19:27:37+02:00,2.0,0.0,2.0,...,Positive,Positive,2020-02-09 19:27:37+02:00,Chingola,Chingola District,Copperbelt Province,Zambia,1,Sunday,en


In [66]:
df_query.text = df_query.apply(lambda row: re.sub(r"http\S+", "", row.text).lower(), 1)
df_query.text = df_query.apply(lambda row: " ".join(filter(lambda x:x[0]!="@", row.text.split())), 1)
df_query.text = df_query.apply(lambda row: " ".join(re.sub("[^a-zA-Z]+", " ", row.text).split()), 1)
timestamps = df_query.date.to_list()
tweets = df_query.text.to_list()

In [67]:
docs = list(map(str,  df_query['prep_text']))
topic_model = BERTopic(language="english", calculate_probabilities=True, verbose=True)
topics, probs = topic_model.fit_transform(docs)

Batches:   0%|          | 0/161 [00:00<?, ?it/s]

2023-10-21 07:33:44,651 - BERTopic - Transformed documents to Embeddings
2023-10-21 07:33:57,444 - BERTopic - Reduced dimensionality
2023-10-21 07:34:00,323 - BERTopic - Clustered reduced embeddings


In [68]:
topic_model.visualize_barchart()

In [69]:
topics_over_time = topic_model.topics_over_time(tweets, timestamps, nr_bins=20)
topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=8)

20it [00:02,  7.94it/s]


In [70]:
sentence_model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = sentence_model.encode(docs, show_progress_bar=False)
reduced_embeddings = UMAP(n_neighbors=10, n_components=2, min_dist=0.0, metric='cosine').fit_transform(embeddings)
topic_model.visualize_documents(docs, reduced_embeddings=reduced_embeddings)

In [71]:
topic_model.visualize_hierarchy(top_n_topics=7)

In [72]:
# Calculate the topic distributions on a token-level
topic_distr, topic_token_distr = topic_model.approximate_distribution(docs, calculate_tokens=True)

# Visualize the token-level distributions
topic_model.visualize_approximate_distribution(docs[1], topic_token_distr[1])

100%|██████████| 6/6 [00:02<00:00,  2.06it/s]


,like,kkia,four,time,big,projected,traffic,zambia,airway,doe,take,like,seven,time,big,term,loan,china,control,come,something,like,15,year
8_loan_debt_china_chinese,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.234,0.525,0.806,1.097,1.038,0.748,0.467,0.176,0.000,0.000,0.000
48_600km_masuwa_exon_unity,0.000,0.000,0.000,0.000,0.000,0.000,0.106,0.106,0.106,0.106,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
63_nail_polish_thumb_remove,0.000,0.116,0.248,0.248,0.248,0.132,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000


In [73]:
pd.DataFrame(topic_model.get_topic(0),columns=["Word","Score"])

,Word,Score
0,zambia,0.020120
1,election,0.019054
2,win,0.018747
3,year,0.018394
4,winner,0.017291
5,excellent,0.015323
6,caf,0.013133
7,next,0.013095
8,zambian,0.012532
9,uganda,0.012496
